In [1]:
#tools
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy import stats
from pandas import DataFrame
import matplotlib.pyplot as plt
import astropy.stats as astat
from solartwins import *
import pickle
from acfunctions import *
import eleanor
from IPython.display import Image
import lightkurve as lk
from starspot import sigma_clipping
import starspot as ss
from lightkurve import search_targetpixelfile
from IPython.display import display, HTML
import starspot.rotation_tools as rt
from scipy import interpolate
import exoplanet
import eleanor
from bokeh.models import ColumnDataSource, LabelSet, Whisker
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.sampledata.autompg import autompg as df

import warnings
warnings.filterwarnings("ignore")

/Users/ilanadoran/opt/anaconda3/lib/python3.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
def find_m_b(x,y,err): #analytical approach to finding line of best fit 
    #C     
    errorsq = np.square(err)
    C = np.diag(errorsq)

    #A 
    xb = ([1] * len(x))
    mata = []
    for z, txt in enumerate(x):
        mata.append(x[z])
        mata.append(xb[z])
    A= np.matrix(mata).reshape((len(x), 2))

    #plugging in 
    At = np.transpose(A)
    invC = np.linalg.inv(C)
    pt1 = np.dot(At, np.dot(invC,A))
    invpt1= np.linalg.inv(pt1)
    pt2 = np.dot(At, np.dot(invC, y)).T
    cov = np.dot(invpt1, pt2)

    m = float(cov[0])
    b = float(cov[1])
    return m ,b 

def residuals(x, y, error): #residual abundance function
    mborig = find_m_b(x, y, error)
    m = mborig[0]
    b = mborig[1]

    predicted_values = [] #y values from slope
    pv = 0
    for u in x:
        pv = (m*u) + b
        predicted_values.append(pv)
        pv = 0

    prev = np.array(predicted_values)
    abu = np.array(y)
    diff = abu - prev #difference between slope and measured values  
    return diff

In [5]:
#dataset with tess vs kepler IDs
tess_kep = pd.read_csv('data_summary.csv')
tess_short = tess_kep[tess_kep.tess_ffi == True]
cat = pd.read_csv('tics_kics_mcq_rot.txt', names=['tic_id', 'kic_id', 'prot'], delim_whitespace=True)

In [6]:
#importing McQ periods
headers = 'TIC ID', 'KIC ID', 'McQ Periods';  

with open("tics_kics_mcq_rot.txt") as file:
    rot_table = [line.split() for line in file.read().splitlines()]

widths = [max(len(value) for value in col)
for col in zip(*(rot_table + [headers]))]

formatting = '{:{widths[0]}}  {:{widths[1]}}  {:{widths[2]}}'
formatting.format(*headers, widths=widths)
for tics_kics_mcq in rot_table:
    formatting.format(*tics_kics_mcq, widths=widths)

In [7]:
# converting txt file data to arrays
tic_ids = []
mcq_periods = []

for index, row in enumerate(rot_table):
    tic_ids.append(int(rot_table[index][0]))
    mcq_periods.append(float(rot_table[index][2]))

In [8]:
# selecting shortest periods only (<13 days, half a tess cycle)
short_periods = []
tics_short_periods = []

for index, period in enumerate(mcq_periods):
    if period < 13: 
        short_periods.append(period)
        tics_short_periods.append(tic_ids[index])

In [9]:
tic_super_short = []
for each, per in enumerate(mcq_periods):
    if per < 6: 
        tic_super_short.append(tic_ids[each])

In [10]:
tess_2m, tess_30m, two_m, thirty_m, errors = [], [], [], [], []

In [11]:
for each, val in tqdm(enumerate(tic_super_short)): 
    star = lk.search_targetpixelfile('TIC' + str(val), mission = 'TESS')
    
    if len(star) > 0: #checking for star in 2min cadence tess databse
        tess_2m.append(val)
        two_m.append(star)

146it [2:45:00, 67.81s/it] 


with open('two_m.data', 'wb') as filehandle:
    pickle.dump(two_m, filehandle)
    
with open('tess_2m.data', 'wb') as filehandle:
    pickle.dump(tess_2m, filehandle)

In [13]:
for ind, each in enumerate(tic_super_short):
    try:
        value = 'TIC' + str(each)
        star_thirty = eleanor.Source(name = value) #checking for star in 30min database
        tess_30m.append(value)
        thirty_m.append(star_thirty)
    except: 
        pass

No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for yo

No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for yo

with open('tess_30m.data', 'wb') as filehandle:
    pickle.dump(tess_30m, filehandle)

In [ ]:
exo_thirty = []

for each, tic in enumerate(thirty_m): 
    data = eleanor.TargetData(tic, do_psf=True, do_pca=True)
    
    q = data.quality == 0
    time = data.time[q] 
    flux_nan = (data.pca_flux[q])[~np.isnan(data.pca_flux[q])]
    norm_flux = flux_nan/np.nanmedian(data.pca_flux[q])
   
    try: 
    #original 
        norm_flux_err = np.sqrt(norm_flux)
        resid = residuals(time, norm_flux, norm_flux_err)
        resid = resid[~np.isnan(resid)]
        time = time[~np.isnan(resid)]   
        
        f = interpolate.interp1d(time, resid)
        xnew = np.arange(time[0], time[-1], 0.1)
        flux = f(xnew) 
        flux_err = np.sqrt(flux)

        #cropping out section
        mask = (time <= 1695) & (time >= 1690)
        norm_flux_2 = norm_flux[~mask]
        time_masked = time[~mask]
    
        mask_2 = (time_masked <= 1709) & (time_masked >= 1704)
        norm_flux_mask = norm_flux_2[~mask_2]
        time_mask = time_masked[~mask_2]

        #removing trends 
        norm_flux_err_mask = np.sqrt(norm_flux_mask)
        resid_mask = residuals(time_mask, norm_flux_mask, norm_flux_err_mask)
        resid_mask = resid_mask[~np.isnan(resid_mask)]
        time_mask = time_mask[~np.isnan(resid_mask)]   
        
        #linear interpolation
        f = interpolate.interp1d(time_mask, resid_mask)
        xnew_mask = np.arange(time_mask[0], time_mask[-1], 0.1)
        flux_mask = f(xnew_mask) 
        flux_err_mask = np.sqrt(flux_mask)
    
        fig, axs = plt.subplots()
        fig.tight_layout(h_pad=6)
        plt.figure(figsize=(15,5))
        plt.plot(xnew, flux, 'royalblue')
        plt.fill_between(xnew, -0.07, 0.03,
                 where = (xnew > 1690) & (xnew <= 1695), 
                 color = 'aliceblue') 
        plt.fill_between(xnew, -0.07, 0.03,
                 where = (xnew > 1704) & (xnew <= 1709), 
                 color = 'aliceblue')

        plt.plot(xnew_mask, flux_mask, 'k')
        fig.savefig('tess_{0}_2m.jpg'.format(each))
        plt.close(fig)

        #exo acf
        ex_1 = exoplanet.autocorr_estimator(xnew_mask, flux_mask, yerr= flux_err_mask, min_period = 0.1, max_period = 15)
        exo_period = ex_1['peaks'][0]['period']
        exo_thirty.append(exo_period)

    except : 
        print('error')
        ls_thirty.append(0)
        pdm_thirty.append(0)
        exo_thirty.append(0)

In [ ]:
zmq.error.ZMQError: Too many open files


with open('exo_thirty.data', 'wb') as filehandle:
    pickle.dump(exo_thirty, filehandle)  

In [ ]:
diff_thirty = []
for each, per in enumerate(exo_thirty):
    tiz = tess_30m[each][3:]
    index = np.where(np.array(tics_short_periods) == np.int(tiz))
    mcq_period = short_periods[index[0][0]]

    diff = np.abs(per - mcq_period)
    diff_thirty.append(diff)

In [ ]:
plt.scatter(exo_thirty, diff_thirty)
plt.ylabel('Difference to McQ')
plt.xlabel('Calculated Period')

In [ ]:
thirty_m_large_diff = []
for each, difference in enumerate(diff_thirty):
    if difference > 2.5:
        print(tess_30m[each])
        thirty_m_large_diff.append(thirty_m[each])

In [ ]:
exo_two = []

for each, tic in enumerate(two_m): 
    #star = lk.search_targetpixelfile('TIC' + str(tic), mission = 'TESS').download();
    #pixelfile = search_targetpixelfile("KIC 8462852", quarter=16).download();
    val = tic.download();
    lc = val.to_lightcurve(aperture_mask='threshold');

    time = lc.time
    flux_nan = (lc.flux[~np.isnan(lc.flux)])
    norm_flux = flux_nan/np.nanmedian(lc.flux)
   
    try: 
    #original 
        norm_flux_err = np.sqrt(norm_flux)
        resid = residuals(time, norm_flux, norm_flux_err)
        resid = resid[~np.isnan(resid)]
        time = time[~np.isnan(resid)]   
        
        f = interpolate.interp1d(time, resid)
        xnew = np.arange(time[0], time[-1], 0.1)
        flux = f(xnew) 
        flux_err = np.sqrt(flux)

        #cropping out section
        mask = (time <= 1695) & (time >= 1690)
        norm_flux_2 = norm_flux[~mask]
        time_masked = time[~mask]
    
        mask_2 = (time_masked <= 1709) & (time_masked >= 1704)
        norm_flux_mask = norm_flux_2[~mask_2]
        time_mask = time_masked[~mask_2]

        #removing trends 
        norm_flux_err_mask = np.sqrt(norm_flux_mask)
        resid_mask = residuals(time_mask, norm_flux_mask, norm_flux_err_mask)
        resid_mask = resid_mask[~np.isnan(resid_mask)]
        time_mask = time_mask[~np.isnan(resid_mask)]   
        
        #linear interpolation
        f = interpolate.interp1d(time_mask, resid_mask)
        xnew_mask = np.arange(time_mask[0], time_mask[-1], 0.1)
        flux_mask = f(xnew_mask) 
        flux_err_mask = np.sqrt(flux_mask)
    
        fig, axs = plt.subplots()
        fig.tight_layout(h_pad=6)
        plt.figure(figsize=(15,5))
        plt.plot(xnew, flux, 'royalblue')
        plt.fill_between(xnew, -0.07, 0.03,
                 where = (xnew > 1690) & (xnew <= 1695), 
                 color = 'aliceblue') 
        plt.fill_between(xnew, -0.07, 0.03,
                 where = (xnew > 1704) & (xnew <= 1709), 
                 color = 'aliceblue')

        plt.plot(xnew_mask, flux_mask, 'k')
        fig.savefig('tess_{0}_2m.jpg'.format(each))
        plt.close(fig)

        #exo acf
        ex_1 = exoplanet.autocorr_estimator(xnew_mask, flux_mask, yerr= flux_err_mask, min_period = 0.1, max_period = 15)
        exo_period = ex_1['peaks'][0]['period']
        exo_two.append(exo_period)

    except : 
        print('error')
        ls_two.append(0)
        pdm_two.append(0)
        exo_two.append(0)

with open('exo_two.data', 'wb') as filehandle:
    pickle.dump(exo_two, filehandle)  

In [ ]:
diff_two = []
for each, per in enumerate(exo_two[0:22]):
    tic = tess_2m[each]
    index = np.where(np.array(tics_short_periods) == tic)
    mcq_period = short_periods[index[0][0]]

    diff = np.abs(per - mcq_period)
    diff_two.append(diff)

In [ ]:
plt.scatter(exo_two[0:22], diff_two)
plt.ylabel('Difference to McQ')
plt.xlabel('Calculated Period')

In [ ]:
two_m_large_diff = []
for each, difference in enumerate(diff_two):
    if difference > 1.5:
        print(tess_2m[each])
        two_m_large_diff.append(two_m[each])

with open('two_m_large_diff.data', 'wb') as filehandle:
    pickle.dump(two_m_large_diff, filehandle)  

In [ ]:
for index, each in enumerate(tess_2m):
    for ind, every in enumerate(tess_30m):
        if int(every[3:]) == each: 
            indx = np.where(np.array(tics_short_periods) == each)
            mcq_period = short_periods[indx[0][0]]
            print(mcq_period, each, np.array(exo_two)[index], exo_thirty[ind], index, ind)